# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096857


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:16,  2.74s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:53,  1.97s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:37,  1.44s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:26,  1.07s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.04it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  3.95it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.13it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  3.51it/s]

Jodelle                               0.001113
Jnursssmith5263                       0.001116
Jonas_Cross                           0.000182
jnarayanbvg                           0.005231
deivad                                0.000255
not-logged-in-c6917d9b19f41e6c2995    0.000782
not-logged-in-906d0937c18c013d2c32    0.145172
TSM_Ginga                             0.000720
framosuw                              0.004012
ComeMyChild                           0.002317
acapirala                             0.000124
helios1235                            0.000113
Lavadude                              0.029950
peakscience7                          0.000141
jordanrushworth                       0.000849
Yalowka                               0.000360
not-logged-in-e5015a872e4a0f43792d    0.003139
FR4N1O                                0.002215
h4ranek                               0.001369
tingard                               0.000304
Fazal.H                               0.001300
ckkmomma     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())